In [ ]:
# Import library yang dibutuhkan
import asyncio
import pandas as pd
from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError
import nest_asyncio

# Terapkan nest_asyncio untuk mendukung environment seperti Jupyter/Colab
nest_asyncio.apply()

# API ID dan Hash Telegram Anda
api_id = "*******"  # Ganti dengan API ID Anda
api_hash = "**********"  # Ganti dengan API Hash Anda

# Nama file sesi
session_name = "tes_telegram"  # Ganti dengan nama sesi Anda

# Fungsi untuk scraping pesan
async def scrape_messages(client, entity_name, limit=100):
    # Dapatkan entitas (group atau channel) dari link atau username
    entity = await client.get_entity(entity_name)

    # Koleksi pesan
    messages = []
    async for message in client.iter_messages(entity, limit=limit):
        # Cek apakah pengirim adalah individu (user) atau sistem
        sender = await message.get_sender() if message.sender_id else None

        # Tentukan nama dan username pengirim berdasarkan jenis entitas
        sender_name = (
            f"{sender.first_name or ''} {sender.last_name or ''}".strip()
            if sender and hasattr(sender, "first_name")
            else "Unknown"
        )
        sender_username = sender.username if sender and hasattr(sender, "username") else "Unknown"

        # Tambahkan pesan ke koleksi
        messages.append({
            'message_id': message.id,
            'date': message.date.strftime('%Y-%m-%d %H:%M:%S'),  # Format tanggal lebih konsisten
            'sender': sender_name,
            'sender_username': sender_username,  # Menambahkan kolom username pengirim
            'message': message.message.strip() if message.message else ''  # Bersihkan teks
        })

    # Simpan data ke dalam DataFrame pandas
    df = pd.DataFrame(messages)

    # Hapus baris dengan pesan kosong
    df = df[df['message'] != '']

    # Tampilkan DataFrame sebelum menyimpan ke file dengan format rapi
    pd.set_option('display.max_colwidth', 200)  # Mengatur lebar kolom pesan agar lebih lebar
    print("\nHasil Pesan yang Diambil (ditampilkan 5 baris pertama):")
    print(df.head())  # Menampilkan beberapa baris pertama dari DataFrame

    # Nama file didasarkan pada link atau username
    output_file = f"{entity_name.replace('https://t.me/', '').replace('/', '_')}_messages.xlsx"

    # Simpan data ke file Excel
    df.to_excel(output_file, index=False, engine='openpyxl')  # Menyimpan sebagai Excel (.xlsx)

    print(f"\nBerhasil menyimpan {len(df)} pesan ke file Excel '{output_file}'")

# Fungsi utama dengan validasi input
async def main():
    # Buat instance client
    async with TelegramClient(session_name, api_id, api_hash) as client:
        try:
            # Masukkan link grup atau channel Telegram
            entity_name = input("Masukkan link grup/channel Telegram (contoh: https://t.me/inicopylinknya): ").strip()
            if not entity_name:
                print("Anda belum memasukkan link. Harap coba lagi.")
                return

            # Tentukan jumlah pesan yang ingin diambil (dengan validasi)
            while True:
                limit_input = input("Masukkan jumlah pesan yang ingin diambil: ").strip()
                if limit_input.isdigit() and int(limit_input) > 0:
                    limit = int(limit_input)
                    break
                else:
                    print("Jumlah pesan harus berupa angka positif.")

            # Panggil fungsi scraping
            await scrape_messages(client, entity_name, limit)

        except SessionPasswordNeededError:
            print("Akun ini membutuhkan verifikasi password 2FA.")
        except Exception as e:
            print(f"Terjadi error: {e}")

# Jalankan loop asyncio secara manual (untuk Jupyter/Colab)
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
